In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
#load csv from WeatherPy
weatherpy_path = os.path.join("..", "WeatherPy", "Output", "city_data.csv")
weatherpy_df = pd.read_csv(weatherpy_path)
weatherpy_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cabo San Lucas,22.89,-109.91,90.0,58,5,10.29,MX,1595027061
1,Tuatapere,-46.13,167.68,39.0,83,39,3.00,NZ,1595027191
2,Souillac,-20.52,57.52,69.8,73,40,11.41,MU,1595027191
3,Mataura,-46.19,168.86,39.0,83,0,3.00,NZ,1595027191
4,Butaritari,3.07,172.79,83.3,76,22,15.64,KI,1595027192


Humidity Heatmap

In [3]:
#configure gmpas
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {'width': '1000px', 'height': '500px', 'border': '1px solid black'}
fig_humidity = gmaps.figure(layout=figure_layout)

#store lat and long as coordinates
coordinates = weatherpy_df[["Lat", "Lng"]]

#store humidity for weight
humidity = weatherpy_df["Humidity"]

#create heatmap
heatmap_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)
fig_humidity.add_layer(heatmap_layer)
fig_humidity


Figure(layout=FigureLayout(border='1px solid black', height='500px', width='1000px'))

In [4]:
#pull locations with ideal weather; temp=55-69, wind speed<5, humidity=40-50%
ideal_weather_df = weatherpy_df.loc[(weatherpy_df["Max Temp"] >=55) & (weatherpy_df["Max Temp"] <=69) & (
                                    weatherpy_df["Wind Speed"] <= 10) & (weatherpy_df["Humidity"] <=50) & (
                                    weatherpy_df["Humidity"] >=40), :]
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Hermanus,-34.42,19.23,60.98,44,0,8.21,ZA,1595027193
76,Bījār,35.87,47.61,63.45,50,0,4.41,IR,1595027203
93,Belaya Gora,68.53,146.42,57.85,50,0,6.33,RU,1595027205
103,Narsaq,60.92,-46.05,64.40,45,0,6.93,GL,1595027207
107,Yellowknife,62.46,-114.35,68.00,40,75,9.17,CA,1595027208
273,Inuvik,68.35,-133.72,64.40,48,75,5.82,CA,1595027243
398,Khovd,48.01,91.64,62.37,49,0,0.67,MN,1595027262


In [5]:
hotel_df = ideal_weather_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

C:\Users\aliss\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Hermanus,-34.42,19.23,60.98,44,0,8.21,ZA,1595027193,
76,Bījār,35.87,47.61,63.45,50,0,4.41,IR,1595027203,
93,Belaya Gora,68.53,146.42,57.85,50,0,6.33,RU,1595027205,
103,Narsaq,60.92,-46.05,64.40,45,0,6.93,GL,1595027207,
107,Yellowknife,62.46,-114.35,68.00,40,75,9.17,CA,1595027208,


In [6]:
parameters = {"radius": 5000, "types": "hotel", "keyword": "hotel", "key": g_key}

# Use the lat/lng to identify nearby hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    parameters["location"] = f"{lat},{lng}"

    # search on type:hotels and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=parameters)

    # convert to json
    name_address = name_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No Hotel Found"
        
#print out updated dataframe with hotel names retrieved
hotel_df

C:\Users\aliss\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Hermanus,-34.42,19.23,60.98,44,0,8.21,ZA,1595027193,One Marine Drive Boutique Hotel
76,Bījār,35.87,47.61,63.45,50,0,4.41,IR,1595027203,Hotel Bame Iran
93,Belaya Gora,68.53,146.42,57.85,50,0,6.33,RU,1595027205,No Hotel Found
103,Narsaq,60.92,-46.05,64.40,45,0,6.93,GL,1595027207,Igdlo Guesthouse
107,Yellowknife,62.46,-114.35,68.00,40,75,9.17,CA,1595027208,The Explorer Hotel
273,Inuvik,68.35,-133.72,64.40,48,75,5.82,CA,1595027243,Arctic Chalet LTD
398,Khovd,48.01,91.64,62.37,49,0,0.67,MN,1595027262,Steppe hotel


In [7]:
#drop city with "No Hotel Found"
hotel_cleaned_df = hotel_df.loc[(hotel_df["Hotel Name"] != "No Hotel Found")]

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_cleaned_df.iterrows()]
hotel_coords = hotel_cleaned_df[["Lat", "Lng"]]

hotel_markers = gmaps.marker_layer(hotel_coords, info_box_content=hotel_info)
fig_humidity.add_layer(hotel_markers)
fig_humidity

Figure(layout=FigureLayout(border='1px solid black', height='500px', width='1000px'))